# Light Switching Optimization

This Notebook contains exploratory code related to developing an optimization model which determines the lane each traffic light opens at a given point in time. Each light determines it's individual optimal behavior. The goal is to maximize the number of cars that can pass an intersection over an undefined period of time. Each light can only open one lane at a time. Lights are further constrained by a cooldown, which limits a light to change it's decision after 5 steps from the previous decision.

In [53]:
import pyoptinterface as poi
from pyoptinterface import highs
import numpy as np

In [71]:
opt_model = highs.Model()

## Environment

The following section creates the environment to optimize.

In [72]:
connected_lanes = ["intersection_4", "intersection_7", "intersection_1"]
connected_lanes

['intersection_4', 'intersection_7', 'intersection_1']

In [73]:
time = range(0, 11)
time

range(0, 11)

In [74]:
int_4_arrivals = np.array([0, 5, 5, 1, 1, 1, 3, 5, 1, 1, 1])
int_7_arrivals = np.array([0, 1, 1, 10, 10, 10, 7, 1, 10, 10, 10])
int_1_arrivals = np.array([0, 10, 10, 1, 1, 1, 1, 10, 1, 1, 1])

sim_env = np.array([int_4_arrivals, int_7_arrivals, int_1_arrivals]).T

sim_env.shape

(11, 3)

In [75]:
sim_env

array([[ 0,  0,  0],
       [ 5,  1, 10],
       [ 5,  1, 10],
       [ 1, 10,  1],
       [ 1, 10,  1],
       [ 1, 10,  1],
       [ 3,  7,  1],
       [ 5,  1, 10],
       [ 1, 10,  1],
       [ 1, 10,  1],
       [ 1, 10,  1]])

## Decision Variables

At each point in time, a light can make a decision for each connected lane.


$$ x_{lt}\in\{0,1\} $$
$$ with: $$
$$ l\in{L}, \quad L=\{Connected Lanes\} $$
$$ t\in{T}, \quad T=\left[ 0;5 \right]

In [76]:
decisions = opt_model.add_variables(
    time, connected_lanes, domain=poi.VariableDomain.Binary
)

Running HiGHS 1.9.0 (git hash: 66f735e): Copyright (c) 2024 HiGHS under MIT licence terms


## Constraints

A light can only have one open lane at any point in time. It can also decide not to open any lane.

$$ \sum_{t\in{T}} \sum_{l\in{L}} x_{l,t} \leq{1} $$

In [77]:
for time_step in time:
    opt_model.add_linear_constraint(
        poi.quicksum(decisions[time_step, lane] for lane in connected_lanes), poi.Leq, 1
    )

A light cannot make a new decision after a cooldown of 5 time steps.

$$ \sum_{l\in{L}} \sum_{t\in{T}}^{0\leq{t}\leq{5}} (x_{l,t-1}-x_{l,t})^{2} \leq{1}$$

In [38]:
# for lane in possible_lanes:
# opt_model.add_linear_constraint(poi.quicksum(((lanes[tick-1, lane] - lanes[tick, lane]) * (lanes[tick-1, lane] - lanes[tick, lane])) for tick in time[1:]), poi.Leq, 1.0)

In [78]:
for decision in decisions:
    opt_model.add_linear_constraint(
        poi.quicksum(
            (
                (decisions[i, decision[1]] - decisions[(i + 1), decision[1]])
                * (decisions[i, decision[1]] - decisions[(i + 1), decision[1]])
            )
            for i in range(max(0, (decision[0] - 5)), decision[0])
        ),
        poi.Leq,
        1.0,
    )

Check for successful creation of constraint objects:

In [79]:
for decision in decisions:
    print(
        poi.quicksum(
            (
                (decisions[i, decision[1]] - decisions[(i + 1), decision[1]])
                * (decisions[i, decision[1]] - decisions[(i + 1), decision[1]])
            )
            for i in range(max(0, (decision[0] - 5)), decision[0])
        ),
        poi.Leq,
        1.0,
    )

<pyoptinterface._src.core_ext.ExprBuilder object at 0x10498fd70> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x10498dd90> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x10498f9b0> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x10498dd90> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x10498f9b0> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x10498dd90> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x10498f9b0> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x10498dd90> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x10498f9b0> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x10498dd90> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object a

Verify that at each step the model can find the previous decision(s):

In [80]:
for decision in decisions:
    print(f"Current Decision: {decision}")
    [
        print(f"Previous Decision: {decisions[i, decision[1]]}")
        for i in range(max(0, (decision[0] - 5)), decision[0])
    ]
    print(100 * "-")

Current Decision: (0, 'intersection_4')
----------------------------------------------------------------------------------------------------
Current Decision: (0, 'intersection_7')
----------------------------------------------------------------------------------------------------
Current Decision: (0, 'intersection_1')
----------------------------------------------------------------------------------------------------
Current Decision: (1, 'intersection_4')
Previous Decision: <pyoptinterface._src.core_ext.VariableIndex object at 0x10496ab70>
----------------------------------------------------------------------------------------------------
Current Decision: (1, 'intersection_7')
Previous Decision: <pyoptinterface._src.core_ext.VariableIndex object at 0x10496bf10>
----------------------------------------------------------------------------------------------------
Current Decision: (1, 'intersection_1')
Previous Decision: <pyoptinterface._src.core_ext.VariableIndex object at 0x1052d4f3

## Objective

The goal is to maximize the number of cars that can pass the light.

In [81]:
objective = poi.quicksum(
    poi.quicksum(
        decisions[time_step, lane] * sim_env[time_step][connected_lanes.index(lane)] for lane in connected_lanes
    )
    for time_step in time
)
opt_model.set_objective(objective, poi.ObjectiveSense.Maximize)

Hessian has dimension 33 but no nonzeros, so is ignored


## Optimization

In [82]:
opt_model.optimize()

Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
10 rows, 30 cols, 30 nonzeros  0s
7 rows, 17 cols, 14 nonzeros  0s
0 rows, 7 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   97              97                 0.00%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      97
  Dual

## Results

Get the number of total cars, that crossed the intersection:

In [83]:
print(f"Objective Value: {opt_model.get_obj_value()}")

Objective Value: 97.0


Get the decision for each point in time:

In [84]:
for lane in decisions:
    if opt_model.get_value(decisions[lane]) == 1:
        print(
            f"Time: {lane[0]}; Intersection: {lane[1]}"
        )

Time: 1; Intersection: intersection_1
Time: 2; Intersection: intersection_1
Time: 3; Intersection: intersection_7
Time: 4; Intersection: intersection_7
Time: 5; Intersection: intersection_7
Time: 6; Intersection: intersection_7
Time: 7; Intersection: intersection_1
Time: 8; Intersection: intersection_7
Time: 9; Intersection: intersection_7
Time: 10; Intersection: intersection_7


Get a matrix of decisions with the lanes as columns and time as rows:

In [85]:
decision_matrix = [opt_model.get_value(decisions[lane]) for lane in decisions]
decision_matrix_time = [
    decision_matrix[i : i + 3] for i in range(0, len(decision_matrix), 3)
]
decision_matrix_time

[[0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0]]

Get a transposed decision matrix:

In [86]:
decision_matrix_light = list(map(list, zip(*decision_matrix_time)))
decision_matrix_light

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0],
 [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]]

For each point in time, get the previous decisions and evaluate if a light can switch:

In [87]:
for light in decision_matrix_light:  # Historie von Entscheidung pro Ampel
    for i in range(len(light)):
        decision_history = (
            [0] * max(0, 5 - i) + light[max(0, i - 5) : i]
        )  # Entscheidungen von t-5 bis t=0; Wenn vorher keine Entscheidung gab, dann führende 0; Zu erkennen an int 0 vs. float 0.0
        block_status = 0  # Bei 0 darf geöffnet werden, bei 1 ist cooldown aktiv, bei größer 1 gabs nen fehler in der entscheidung
        for j in range(1, (len(decision_history) - 1)):
            block_status += (
                decision_history[j - 1] - decision_history[j]
            ) ** 2  # Berechnen von @mxrio Loop (** ist exponent)
        print(decision_history)
        print(block_status)
        if block_status == 0:
            print("Light can switch")
        elif block_status == 1:
            print("Light is cooldown blocked")
        elif block_status > 1:
            print("Forbidden Action")

[0, 0, 0, 0, 0]
0
Light can switch
[0, 0, 0, 0, 0.0]
0
Light can switch
[0, 0, 0, 0.0, 0.0]
0.0
Light can switch
[0, 0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0, 0, 0, 0, 0]
0
Light can switch
[0, 0, 0, 0, 0.0]
0
Light can switch
[0, 0, 0, 0.0, 0.0]
0.0
Light can switch
[0, 0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0, 0.0, 0.0, 0.0, 1.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 1.0, 1.0]
1.0
Light is cooldown blocked
[0.0, 0.0, 1.0, 1.0, 1.0]
1.0
Light is cooldown blocked
[0.0, 1.0, 1.0, 1.0, 1.0]
1.0
Light is cooldown blocked
[1.0, 1.0, 1.0, 1.0, 0.0]
0.0
Light can switch
[1.0, 1.0, 1.0, 0.0, 1.0]
1.0
Light is cooldown blocked
[1.0, 1.0, 0.0, 1.0, 1.0]
2.0
Forbidden Actio